# 1. SQL Server'da Index Kavramı
* Bir veritabanı yönetim sisteminin bilinmesi gereken en önemli özelliklerinden biridir. **Performansa etkisi çok fazladır**.
* Veritabanı yönetim sistemlerinin temel misyonu veriyi yönetmektir. Veriyi okumak, yazmak.
* Genel ortalamada veritabanı yönetim sistemlerinde %95 okuma, %5 yazma yapılır. IoT'de yazma fazladır.
* Bu durumda okuma performansını arttıracak her türlü işlem veritabanı sunucunun performansının arttırılmasında doğrudan etkili olur.

## Rehber Örneği
* Rehberler alfabetik sıraya göre A'dan Z'ye sıralanmaktadır.
* Aranmak istenen isim seçildiğinde rehberin sayfa sayısına göre tam ortadan rehber kesilir.
* Aranan isim sayfanın başındaki kişiyle karşılaştırıldığında büyük veya küçüklük durumuna göre rehberin yarısı atılır.
* Bu işlem ortadan bölerek devam eder. 
* SQL Serverdaki index mantığı bu şekilde çalışır.

# 2. Excel Üzerinde Arama 1
* Sıralamak istediğin sütunu seçip *Filter* ile sıralama yapabilirsin.
* Sütunu sıraladıktan sonra rehber örneğindeki gibi iterasyon yapılarak örneğin 1000 satırlık bir veriden bir ismi 9 adımda çekebilirsin.
* Bu algoritmaya **binary tree** denir.

# 3. Excel Üzerinde Arama 2
* Aramayı gerçekleştirmek için sıralama yaptığın sayfa index sayfası olur.
* Bulduğun sonucu ID ile eşleştirerek ID'ye göre sıralı data sayfasından arama yapabilirsin.
* Hızlı arama index sayfasından gerçekleştirilir, veri data sayfasından getirilir. Bu işlemin teknik adı **page lookup**dır.
* Eğer farklı bir sütundaki farklı bir değere göre veriler aranacaksa da o sütuna page lookup yapılır.
* Bir tabloda birden fazla index olabilir. 
* Tablodaki satırı tanımlayan ve primary key olan sütun **clustered index** anlamına gelir.
* **Clustered index, veriyi sql'de fiziksel olarak sıraya sokan yapıdır.**
* **Non-clustered index**, page lookup yapmak amacıyla elde edilen sütunların indexli yapısına denir.

# 4. Büyüyen Veride Performans Hesaplama

<img src = "https://i.hizliresim.com/8lznjg4.png" width = 700>

* 5000 satırlık bir veride aranılacak satır index sayesinde en fazla 11 adımda bulunur.

<img src = "https://i.hizliresim.com/t2im6yl.png" width = 700>

* Veri 2 katına çıktığı takdirde indexli arama sadece 1 adım yavaşlar.

<img src = "https://i.hizliresim.com/1sm10r8.png" width = 700>

* Index olmasa 200.000 kat fazla işlemle bulunacak sonuç index ile sadece 3 kat daha fazla işlemle bulunur.

* **Index gerçek performansını büyük veride gösterir.**

* **Indexleme okumayı hızlandırır.**

* **Doğru indexleme yapılırsa performans 1 milyar veride yaklaşık 30 milyon kat daha hızlı okuma yapılabilir.**


# 5. Index Uygulama
* Okuma işleminin nasıl yapıldığını *Display Estimated Execution Plan* sekmesinden öğrenebilirsin.

### Table Scan
<img src = "https://i.hizliresim.com/itbxpdd.png" width = 700>

* Table Scan, kayıtların karışık pagelerde tutulduğu ve pagelere tek tek bakılıp kayıtların tek tek arandığı en kötü, en verimsiz arama yöntemidir.
* Eğer tabloda PRIMARY KEY yoksa bu tarama yapılır.

### Clustered Index Scan
<img src = "https://i.hizliresim.com/jcfmaae.png" width = 700>

* Tabloda PRIMARY KEY varsa bu tarama yapılır.
* Clustered Index, fiziken tablodaki veridelerin sıralanma ve yazılma şeklidir.
* Her satırı tek tek kontrol eder.

#### <CODE>SET STATISTICS IO ON</CODE>
* Okuma sayısının öğrenilebileceğin komut.
* Sorgu yapıldıktan sonra *Messages* kısmından *logical reads* bölümü bize okumanın yapılma sayısını gösterir. Bu sayı okunan page sayısıdır.
* Bir page 8 KB olduğundan *logical reads* ile çarpımı okumanın maaliyetini verir.

#### <CODE>SP_SPACEUSED 'TABLOADI'</CODE>
* Tablonun kapladığı alanı ve satır sayısını bulabilirsin.

### Non-clustered Index Atama
* **WHERE** ile veri çektiğinde o verinin daha hızlı gelmesini sağlayacak öneriyi yine *Display Estimated Execution Plan* sekmesinde bulabilirsin.
* Non-clustered index oluşturma scriptini çalıştırdığın takdirde okuma işlemi daha hızlı yapılır.

* Tablonun *Indexes* kısmından eklenilebileceği gibi kendi scriptimizi yazarak da eklenebilr.

<img src = "https://i.hizliresim.com/lzjqyms.png">

# 6. Included Column

<img src = "https://i.hizliresim.com/nuw40mx.png">

### Index Seek
* Binary tree yapısına göre yapılan aramanın adıdır.

### Key Lookup
* Indexe göre veri bulunduktan sonra ID'ye göre verinin getirilmesi için kullanılan yapı.

#### Sadece NAMESURNAME sütununu getirdiğimizde zaten indexleme o sütuna göre yapıldığı için sadece Index Seek işlemi yapılır.

<img src = "https://i.hizliresim.com/25q0z00.png">

* *Index Seek* ve *Key Lookup* işlemlerinin yapılması iki kere işlem yapıldığı için can sıkıcı olabilir. 
* Bunu daha hızlı çalıştırmak için **Included Column** kullanılabilir.

### Included Column
* ID ve indexlenecek sütunun haricinde diğer sütunlarında bulunduğu index yapısına **Included Column** denir.
* Nonclustered index oluşturulurken **Included Column** sekmesinde diğer sütunlar seçilerek işlem gerçekleştirilir.

<img src = "https://i.hizliresim.com/5zy21hu.png">

* Sonuçta *logical reads* değeri düşer. Ancak bunun da bir maaliyeti vardır.

<img src = "https://i.hizliresim.com/48sp5as.png">

* Sadece primary key için index alanı kullanıldığında 408 KBlık küçük bir alan kullanılır.
* Nonclustered index kullanıldığında bu alan 69560 KB'a çıkarken (ekstra olarak ID alanı ve ilgili kolon eklenir), included column eklendiğinde alan neredeyse veriye eşit olur.

# 7. Birden Fazla Index Kullanma
* Yaptığın sorgulardan ulaştığın indexleme tavsiyelerine göre yeni indexler oluşturabilirsin.
* Birden fazla ifadenin geçtiği sorgularda hangi indexlemenin kullanılacağına sistem karar verir. Maaliyeti düşük olan yol seçilir.
* **AND** sorgusunda 1 tane index kullanıldı. **OR**'da 2 tane.
* Indexin değeri tarih gibi birden fazla kopyası bulunabilen bir değer olduğunda indexleme önerisi **included** şeklinde olur. Key lookup yapılmaz.
* Birden fazla ifadenin geçtiği sorgularda daha az kayıt getiren ifadenin indexi kullanılır.

# 8. Unique Index
* SQL Serverda indexin unique olduğu belirtilmediği takdirde satırı bulsa bile başka var mı diye aramaya devam eder.
* Indexi tanımlarken unique olduğu belirtilebilir. Unique olarak belirtildikten sonra yeni bir kayıt eklendiği veya bir kayıt değiştirildiği takdirde önceden bulunan değerlerden biri kullanılamaz.
* Ayrıca aramada bulunduktan sonra aramaya devam edilmez.
* **Unique olan sütunlarını unique index olarak ayarla.**

# 9. Index Bozulmaları
* Gerçekte data statik değildir, sürekli yeni kayıtlar eklenmektedir.
* Veri fiziksel olarak primary keye göre yazılır.
* Clustered index, bozulmasın diye, özellikle otomatik artan olarak verilir.
* Sonradan eklenen kayıt otomatik IDsi ile eklendiği için en sona eklenir ve sıralama hiç bozulmaz.

<img src = "https://i.hizliresim.com/fkut0ja.png" width = 500>

---------

* Index tarafında ise index oluşturulduğunda, index alanına göre sıralama yapılır.

<img src = "https://i.hizliresim.com/sn4m1ym.png" width = 500>

* Tablolar geri tarafta pageler halinde tutulur.
* Binlerce olan satır 8 KBlık pagelere bölünmüş şekilde tutulur.

<img src = "https://i.hizliresim.com/hrct26m.png" width =700>

* Kayıtın boyutuna göre bir pagede farklı sayıda kayıt olabilir.
* Yeni bir kayıt geldiğinde yeni bir page açılır ve kayıtlar yeni page'e eklenmeye devam eder.

<img src = "https://i.hizliresim.com/k2noyvg.png" width = 700>

* Index pagelerinde ise yine yeni kayıtlar yeni pagelere eklenir ancak sıralanması indexi bozuk ve sırasız bir şekilde eklenir.
* Indexlerin önceden sıralanmış halinde indexler düzgün iken yeni eklenen kayıtlar fragmantasyona sebebiyet verir.
* Indexler her kayıt eklendiğinde bozulur ancak bu bozulmanın süresini uzatmanın da bir yolu vardır. Bu süreyi uzatmanın nedeni sistemlerin 7/24 açık olmalarından kaynaklanır. Her fragmantasyonda bozulma yaşanabileceği için rebuild süresi uzatılır.


<img src = "https://i.hizliresim.com/r0vforl.png" width = 700>

* Pageler içinde fill factor denilen boşluklar oluşturularak yeni kayıtların gelmesi durumunda page içinde minik kaydırmalar ile veriler indexleri bozulmadan eklenir.
* Bu süreç fill factorun doluluğa erişmesiyle son bulur ve index bozulur ancak o zamana kadar bize bakım için zaman kazandırır.
* **Indexler satır satır oluşturulmaz page page oluşturulur.**

# 10. Index Bozulmaları Uygulama
* Fragmantasyonun ortadan kaldırılıp indexlerin düzeltilmesine **rebuild** denir.
* Fragmantasyon ilgili indexin *Properties > Fragmentation* sekmesinden görülebilir.
* Indexleme yeni yapılmışken bu değer 0'lar civarında iken eklenen kayıt sayısına göre artarak değişkenlik gösterir.
* Properties kısmından minimum row size ve maximum row size kısımlarından bir pagein tuttuğu minimum ve maksimum satır sayısını görüntüleyebilirsin.
* Eğer %40 üstüne çıkarsa bu değer indexler kesinlikle düzeltilmelidir.
* Fragmantasyon durumunda arama yaparken sistem bir yere kadar sıralı olarak bakar ancak bir süre sonra sırasız olarak bakmaya devam eder.

------

* Bir indexi düzeltmenin iki yolu vardır.
    * **Rebuild**
    * **Reorganize**
* Aralarındaki fark şudur;
    * Eğer masanın üzerinde bir sürü kağıt olduğunu düşünürsek ve bu kayıtlar dağınık haldeyse toparlanması gerekiyorsa;
    * Eğer masanın üzerinde hallediliyorsa bu düzeltme işlemi; **reorganize**,
    * Eğer masanın üzeri boşaltılıp yerden tek tek kağıtlar düzeltilerek koyuluyorsa bu düzeltme işlemi; **rebuild** olarak adlandırılır.
   
#### Fragmantasyon %40'ın altındaysa *reorganize*, %40'ün üzerindeyse *rebuild* ama sen hep rebuild yap.   
------
#### Fill factor

* Ancak buna gerek kalmadan ya da daha ileri bir zamanda yapmak için *Options > Fill factor* alanına doluluk oranı gir.
* Eğer 70 girersen pagein %70ini doldurur %30unu yeni gelecek kayıtlar için boş tutar.
* Index bozulmalarının önüne geçilir ve performans olarak geri döner.

-------

* Eğer fillfactor default olarak her indexlemede aynı değeri alsın istiyorsan *Server > Properties > Database Settings*'den ayarlayabilirsin.

# 11. İstatistikler
* SQL Server performansında önemli bir yere sahiptir.
* Eğer iki farklı kolon için indexleme yapmışsan ve bu iki kolonu where şartında AND ile getiriyorsan hangi şartta daha az kayıt varsa o indexi kullanır.

<img src = "https://i.hizliresim.com/9oh4aj3.png">

* Peki SQL Server IX1'i kullanacağını yani o indexin kolonundan daha az kayıt geleceğini nereden biliyor? Burada istatistikler devreye girer.
* **Statistics**, Indexes klasörünün altında tutulur ve her index için bulunur.
* Her bir index tanımlandığında veya bozulmadan rebuild edildiğinde istatistikler güncellenir.
* Eğer istatistikler güncellenmezse sorgu hala daha az maliyetli olacağını bildiği indexi kullanır.
* **Eğer sorgunun istediğin indexini kullanmasını istiyorsan WITH ile belirtirsin ancak pek önerilmez.**

<img src = "https://i.hizliresim.com/4u4jwio.png" width = 500>

### İstatistik güncellemenin üç yolu vardır;

#### 1. Index rebuild veya reorganize edilirse istatistik güncellenir.
* Ancak rebuild ve reorganize her zaman yapabileceğin işler değil.

#### 2. <code>SP_UPDATESTATS</code>
* Veri tabanındaki tüm tablolar için istatistikleri günceller.

#### 3. <code>UPDATE STATISTICS CUSTOMERS</code>
* Sadece update edilmek istenen tablonun istatistiklerini günceller.

#### İSTATİSTİKLERİN HER GÜN GÜNCELLENMESİ GEREKİR!